# Elcom Netzbetreibertarife abrufen via Linked Data Services LINDAS

Die Linked Data Services Platform des Bundes, kurz LINDAS, erlaubt es, mittels der query-sprache SPARQL Daten abzufragen. Daten aus unterschiedlichen Datenbanken können ohne expliziten JOIN kombiniert und verbunden werden.

Das nachfolgende Jupyter-Notebook bietet einen kleinen Überblick und code snippets zur Durchführung einfacher abfragen.

Dieses Notebook könnte man als leich abgeänderte und verkürzte Versions des Notebooks der Firma Zazuko betrachten, welche unter folgendem Link auffindbar ist: [Link zum Zazuko Notebook](https://jupyter.zazuko.com/electricity_prices.html).

### Packages installieren
Um die LINDAS Datenbank via Python zu erreichen, bedarf es folgenden Python Packages
* Pandas (für die Datenverarbeitung)
* SparqlClient von graphly (um SPARQL abfragen zu schreiben)

Der SparqlClient ist auf GitHub zu finden (https://github.com/zazuko/graphly) und kann mittels folgendem commmand im terminal installiert werden: `pip install git+https://github.com/zazuko/graphly`

In [1]:
!pip install pandas
!pip install git+https://github.com/zazuko/graphly.git

  Cloning https://github.com/zazuko/graphly.git to /private/var/folders/x5/3_ztps2d5gn3l472r72dmmn80000gn/T/pip-req-build-4mq1s4jl
  Running command git clone --filter=blob:none --quiet https://github.com/zazuko/graphly.git /private/var/folders/x5/3_ztps2d5gn3l472r72dmmn80000gn/T/pip-req-build-4mq1s4jl
  Resolved https://github.com/zazuko/graphly.git to commit 4e7ca9987ab08f6e11ced977e003c244832ef42b
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 15.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 16.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00
  Created wheel for graphly: filename=graphly-0.1-py3-none-any.whl size=5206 sha256=3c3db0d67bf7f7b960d2650202b10b9343c276114795a22f9e9c8ea0d828bead
  Stored in directory: /private/var/folders/x5/3_ztps2d5gn3l472r72dmmn80000gn/T/pip-ephem-wheel-cache-dxylh604/wheels/a4/d3/ab/7ea4d6c4755bb0610ab0459a82f

Nach erfolgreicher installation können die Packages importiert werden:

In [2]:
import pandas as pd
from graphly.api_client import SparqlClient

LINDAS entspricht der RDF-Architektur, wobei RDF für Resource Description Framework steht. Für die SPARQL Abfrage werden tripels bestehend aus Subjekt, Prädikat und Objekt deklariert, die dieses Muster in der verlinkten Datenstruktur angleichen. Einen guten Einstieg in linked data und SPARQL bietet das folgende video: https://www.youtube.com/watch?v=FvGndkpa4K0

Im folgenden werden ein paar queries aufgezeigt, die in Zukunft hilfreich sein könnten. Als erstes muss eine Sparql-Client Instanz mit einem endpoint definiert werden, um die Abfragen durchzuführen zu können.

In [3]:
sparql = SparqlClient("https://lindas.admin.ch/query")
geosparql = SparqlClient("https://geo.ld.admin.ch/query")

sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "cube": "<https://cube.link/>",
    "elcom": "<https://energy.ld.admin.ch/elcom/electricityprice/dimension/>",
    "admin": "<https://schema.ld.admin.ch/>"
})

geosparql.add_prefixes({
    "dct": "<http://purl.org/dc/terms/>",
    "geonames": "<http://www.geonames.org/ontology#>",
    "schema": "<http://schema.org/>",
    "geosparql": "<http://www.opengis.net/ont/geosparql#>",
})

### Liste aller Netzbetreiber
Im Folgenden eine vollständige Liste aller Netzbetreiber gemässe elcom. Unter dem folgenden [Link](https://s.zazuko.com/wE57VK) kann die Abfrage direkt als SPARQL auf der LINDAS-homepage durchgeführt werden.

In [4]:
query1 = f"""
  SELECT DISTINCT ?operatorName
  FROM <https://lindas.admin.ch/elcom/electricityprice>
  WHERE {{

    <https://energy.ld.admin.ch/elcom/electricityprice> cube:observationSet/cube:observation ?obs .

    ?obs a cube:Observation ;
      elcom:period "2025"^^xsd:gYear ; # data available up to 10 years ago
      # elcom:municipality ?muniri ;
      elcom:category/schema:name ?category;
      elcom:operator ?operator .

    ?operator schema:name ?operatorName .

    FILTER( ?category = "C3")

  }}
  """
operatorNames = pd.DataFrame()
operatorNames = sparql.send_query(query1)
operatorNames.head()

,operatorName
0,Elektrizitätswerke des Kantons Zürich (EKZ)
1,"Gemeinde Andelfingen, Stromversorgung"
2,Elektrizitätsgenossenschaft Marthalen
3,"Politische Gemeinde Dietlikon, Gemeindewerke"
4,Genossenschaft Licht- und Kraftwerke Glattfelden


### Liste aller Gemeinden und ihre jeweiligen Netzbetreiber
Im Folgenden eine Abfrage der jeweiligen Gemeinden und entsprechende Netzbetreiber. Unter dem folgenden [Link](https://s.zazuko.com/ZEvzwf) kann die Abfrage direkt als SPARQL auf der LINDAS-homepage durchgeführt werden.

In [19]:
query2 = """
    PREFIX schema: <http://schema.org/>
    PREFIX cube: <https://cube.link/>
    PREFIX elcom: <https://energy.ld.admin.ch/elcom/electricityprice/dimension/>
    PREFIX admin: <https://schema.ld.admin.ch/>

    SELECT ?municipality_id (2025 as ?period) ?operatorName ?category ?energy ?grid ?aidfee (?community_fees + ?aidfee as ?taxes) ?fixcosts ?variablecosts
    FROM <https://lindas.admin.ch/elcom/electricityprice>
    WHERE {
        <https://energy.ld.admin.ch/elcom/electricityprice/observation/> cube:observation ?observation.

        ?observation
        elcom:category/schema:name ?category;
        elcom:municipality ?municipality_id;
        elcom:period "2025"^^<http://www.w3.org/2001/XMLSchema#gYear>;
        elcom:product <https://energy.ld.admin.ch/elcom/electricityprice/product/standard>;
        elcom:fixcosts ?fixcosts;
        elcom:total ?variablecosts;
        elcom:operator ?operator ;
        elcom:gridusage ?grid;
        elcom:energy ?energy;
        elcom:charge ?community_fees;
        elcom:aidfee ?aidfee.

        # Fetch operator information
        ?operator schema:name ?operatorName .

        # Filter for Zürich (assuming Bern has a known municipality_id)
        FILTER (?municipality_id = <https://ld.admin.ch/municipality/191>)  # Replace with the actual URI for Bern
    }
    ORDER BY ?municipality_id ?category ?variablecosts
    LIMIT 20
  """
operatorNames = pd.DataFrame()
operatorNames = sparql.send_query(query2)
operatorNames.head(20)

,municipality_id,period,operatorName,category,energy,grid,aidfee,taxes,fixcosts,variablecosts
0,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,C1,15.321,11.125,2.3,2.3,90.0,28.746
1,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,C2,15.389,10.252,2.3,2.3,90.0,27.941
2,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,C3,14.907,9.704,2.3,2.3,240.0,26.911
3,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,C4,14.835,9.097,2.3,2.3,240.0,26.233
4,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,C5,14.835,6.531,2.3,2.3,360.0,23.667
5,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,C6,14.273,6.114,2.3,2.3,360.0,22.687
6,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,C7,14.172,5.477,2.3,2.3,360.0,21.950
7,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,H1,15.092,14.831,2.3,2.3,90.0,32.223
8,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,H2,15.064,12.755,2.3,2.3,90.0,30.119
9,https://ld.admin.ch/municipality/191,2025,Glattwerk AG,H3,14.771,10.544,2.3,2.3,90.0,27.615


### Liste alles Addressen
Im Folgenden eine Abfrage die alle Addressen der Schweiz retourniert, sowie die position als Punkt-Geometrie. Unter dem folgenden [Link](https://s.zazuko.com/BULwVM) kann die Abfrage direkt als SPARQL auf der LINDAS-homepage durchgeführt werden.

In [20]:
query3 = """
    PREFIX schema: <http://schema.org/>
    PREFIX gn: <http://www.geonames.org/ontology#>
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>
    PREFIX dct: <http://purl.org/dc/terms/>

    SELECT ?address ?street ?locality ?postalCode ?geometry
    WHERE {
    ?address a schema:PostalAddress ;
           schema:streetAddress ?street ;
           schema:addressLocality ?locality ;
           schema:postalCode ?postalCode ;
           geo:hasGeometry ?geometry .
    }
    LIMIT 100
    """
addresses = pd.DataFrame()
addresses = geosparql.send_query(query3)
addresses.head(20)

,address,street,locality,postalCode,geometry
0,https://geo.ld.admin.ch/location/address/10190...,Rennweg / Chemin de la Course 47,Biel/Bienne,2504 Biel/Bienne,https://geo.ld.admin.ch/location/address/geome...
1,https://geo.ld.admin.ch/location/address/10004...,Grossweid 25,Seegräben,8607 Aathal-Seegräben,https://geo.ld.admin.ch/location/address/geome...
2,https://geo.ld.admin.ch/location/address/10044...,Tüftelstrasse 53,Koblenz,5322 Koblenz,https://geo.ld.admin.ch/location/address/geome...
3,https://geo.ld.admin.ch/location/address/10004...,Hertistrasse 19,Gossau (ZH),8614 Bertschikon (Gossau ZH),https://geo.ld.admin.ch/location/address/geome...
4,https://geo.ld.admin.ch/location/address/10001...,Leeweg 17,Bülach,8180 Bülach,https://geo.ld.admin.ch/location/address/geome...
5,https://geo.ld.admin.ch/location/address/10007...,Dorfstrasse 124,Meilen,8706 Meilen,https://geo.ld.admin.ch/location/address/geome...
6,https://geo.ld.admin.ch/location/address/10004...,Schönaustrasse 48,Bäretswil,8344 Bäretswil,https://geo.ld.admin.ch/location/address/geome...
7,https://geo.ld.admin.ch/location/address/10005...,Allmendgütlistrasse 19,Horgen,8810 Horgen,https://geo.ld.admin.ch/location/address/geome...
8,https://geo.ld.admin.ch/location/address/10007...,Wetzikerstrasse 112,Hittnau,8335 Hittnau,https://geo.ld.admin.ch/location/address/geome...
9,https://geo.ld.admin.ch/location/address/10007...,Huebacherweg 21,Hittnau,8335 Hittnau,https://geo.ld.admin.ch/location/address/geome...


### Gemeinde, Netzbetreiber und C3/H4 Tarife
Im Folgenden die Abfrage nach Gemeinden, Netzbetreiber und Energietarife, gefiltert nach C3 und H4. Unter dem folgenden [Link](https://s.zazuko.com/Uc115h) kann die Abfrage direkt als SPARQL auf der LINDAS-homepage durchgeführt werden.
Code wurde grosszügig übernommen vom [Zazuko-Notebook](https://jupyter.zazuko.com/electricity_prices.html) mit kleinen Abänderungen. 

In [21]:
query4 = """
    PREFIX schema: <http://schema.org/>
    PREFIX cube: <https://cube.link/>
    PREFIX elcom: <https://energy.ld.admin.ch/elcom/electricityprice/dimension/>
    PREFIX admin: <https://schema.ld.admin.ch/>

    SELECT ?municipality_id ?postal_code (2025 as ?period) ?operatorName ?category ?energy ?grid ?aidfee (?community_fees + ?aidfee as ?taxes) ?fixcosts ?variablecosts
    FROM <https://lindas.admin.ch/elcom/electricityprice>
    WHERE {
    <https://energy.ld.admin.ch/elcom/electricityprice/observation/> cube:observation ?observation.

    ?observation
      elcom:category/schema:name ?category;
      elcom:municipality ?municipality_id;
      elcom:period "2025"^^<http://www.w3.org/2001/XMLSchema#gYear>;
      elcom:product <https://energy.ld.admin.ch/elcom/electricityprice/product/standard>;
      elcom:fixcosts ?fixcosts;
      elcom:total ?variablecosts;
      elcom:operator ?operator ;
      elcom:gridusage ?grid;
      elcom:energy ?energy;
      elcom:charge ?community_fees;
      elcom:aidfee ?aidfee.

    # Fetch operator information
    ?operator schema:name ?operatorName .

    # Filter for Bern (assuming Bern has a known municipality_id)
    FILTER (?municipality_id = <https://ld.admin.ch/municipality/1>)  # Replace with the actual URI for Bern
    FILTER( ?category = "C3" || ?category = "H4")
    }
ORDER BY ?municipality_id ?category ?variablecosts
LIMIT 20
  """
operatorNames = pd.DataFrame()
operatorNames = sparql.send_query(query4)
operatorNames.head(20)

,municipality_id,postal_code,period,operatorName,category,energy,grid,aidfee,taxes,fixcosts,variablecosts
0,https://ld.admin.ch/municipality/1,None,2025,Elektrizitätswerke des Kantons Zürich (EKZ),C3,15.312,7.760,2.3,2.46,720.0,25.532
1,https://ld.admin.ch/municipality/1,None,2025,Elektrizitätswerke des Kantons Zürich (EKZ),H4,15.732,9.313,2.3,2.46,60.0,27.506
